![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2021-2022 </font>
</div>

    
<div align="center"> 
<font size=4> Ejercicio C - Calibración de los SLM </font>
</div>

- **Fecha**: *
        
- **Alumno**: *  


# Introducción

En este ejercicio vamos a trabajar con la calibración del SLM para encontrar unas condiciones de modulación de amplitud y modulación de fase. Para ello, hay que diseñar una función de mérito para cada una de los dos tipos de modulación y encontrar una configuración de ángulos para cada una mediante una optimización.

## Cargar calibración del SLM

Hay que realizar estos pasos para cargar la calibración como un objeto *Jones_vector* de *pypol*.

In [ ]:
import numpy as np
from py_pol.jones_matrix import Jones_matrix
from py_pol.jones_vector import Jones_vector

In [ ]:
# Cargar datos del archivo NPZ (grabado con np.savez)
data = np.load("SLM_Jones_components.npz")
# Definir una lista con las componentes de la matriz de Jones: J00, J01, J10 y J11
components = [data["J0"] * np.exp(1j * data["d0"]),
             data["J1"] * np.exp(1j * data["d1"]),
             data["J2"] * np.exp(1j * data["d2"]),
             data["J3"] * np.exp(1j * data["d3"])]
# Crear el objeto py_pol
Jslm = Jones_matrix()
Jslm.from_components(components);

## Sistema

El sistema será igual que en el laboratorio: un haz inicial de polarización circular a derechas, un generador de estados (PSG) formado por un polarizador perfecto y una lámina de cuarto de onda que pueden rotar, el SLM, y un analizador de estados (PSA) formado por una lámina de cuarto de onda y un polarizador perfecto que pueden rotar.

# Calibración completa

Vamos a calibrar la matriz del SLM que tenemos (como si no supiéramos acceder a sus datos) mediante el método del retardador o el método extendido. Aquellos alumnos que hayan realizado la práctica 4A en el laboratorio deben utilizar el método del retardador (para no repetir lo que hicieron en el laboratorio), mientras que los que realicen la práctica $B deben escoger el método extendido (que es más completo).

## Cálculo de los ángulos

Cada una de las configuraciones en que se van a hacer medidas de intensidad requieren que el PSG y el PSA tengan un estado generado y analizado determinado. Por lo tanto, en primer lugar hay que calcular los ángulos a los que deben colocarse los elementos del PSG y del PSA.

## Medida de la matriz de Jones del SLM

Simula las diferentes medidas de la intensidad y calcula de la matriz de Jones del SLM.

## Comparación 

Compara el módulo y la fase global de los elementos de matriz obtenidos "experimentalmente" con los de la matriz Jslm original.

## Discusión

Discute el resultado obtenido.